In [11]:
import math
import random
import pandas as pd
import numpy as np
import sklearn.cross_validation

In [12]:
from sklearn.datasets import load_boston
boston = load_boston()
x = np.array(boston['data'])
y = np.array(boston['target'])
x_train, x_test, y_train, y_test = sklearn.cross_validation.train_test_split(x, y, test_size = .20, random_state=42)

In [13]:
class Tree(object):
    def __init__(self, parents=None):
        self.children = []
        self.split_feature = None
        self.split_feature_value = None
        self.parents = parents
        self.label = None

def std_reduction(y_train):
    return -np.std(y_train)

def split_data(x_train, y_train, feature_index):
    attribute_values = x_train[:,feature_index]
    for attribute in set(attribute_values):
        data_subset = []
        for index, point in enumerate(x_train):
            if point[feature_index] == attribute:
                data_subset.append([point, y_train[index]])
        yield data_subset

def gain(x_train, y_train, feature_index):
    reduction = std_reduction(y_train)
    for data_subset in split_data(x_train, y_train, feature_index):
        reduction -= std_reduction([label
                    for (point, label) in data_subset])
    return reduction

def homogeneous(y_train):
    return len(set(y_train)) <= 1

def majority_vote(y_train, node):
    labels = y_train
    choice = max(set(labels), key=list(labels).count)
    node.label = choice
    return node

def build_decision_tree(x_train, y_train, root, remaining_features):
    remaining_features = np.array(list(remaining_features))
    if homogeneous(y_train):
        root.label = y_train[0]
        return root
    
    if remaining_features.shape == 0:
        return majority_vote(y_train, root)
    
    indices = np.random.choice(int(remaining_features.shape[0]), int(2*remaining_features.shape[0]/3), replace = False)

    best_feature = max(remaining_features[indices], key=lambda index: 
                       gain(x_train, y_train, index))
    remaining_features = set(remaining_features)
    if gain(x_train, y_train, best_feature) == 0:
        return majority_vote(y_train, root)
    
    root.split_feature = best_feature
    
    for data_subset in split_data(x_train, y_train, best_feature):
        child = Tree(parents = root)
        child.split_feature_value = data_subset[0][0][best_feature]
        root.children.append(child)
        
        new_x = np.array([point for (point, label) in data_subset])
        new_y = np.array([label for (point, label) in data_subset])
        
        build_decision_tree(new_x, new_y, child, remaining_features - set([best_feature]))
    
    return root

def decision_tree(x_train, y_train):
    return build_decision_tree(x_train, y_train, Tree(), 
                               set(range(len(x_train[0]))))
def find_nearest(array, value):
    nearest = (np.abs(array-value)).argmin()
    return array[nearest]

def evaluate(tree, point):
    if tree.children == []:
#         print "label = %r" %(tree.label)
        return tree.label
    else:
        try:
            matching_children = [child for child in tree.children
                if child.split_feature_value == point[tree.split_feature]]
            return evaluate(matching_children[0], point)
        except:
            array = [child.split_feature_value for child in tree.children]
            point[tree.split_feature] = find_nearest(array, point[tree.split_feature])
            matching_children = [child for child in tree.children
                if child.split_feature_value == point[tree.split_feature]]
            return evaluate(matching_children[0], point)
        
def predict(x_test, tree):
    predicted_labels = [evaluate(tree, point) for point in x_test]
    return predicted_labels

def random_forest(x_train, y_train, x_test, n_estimators = 100):


    x_train_copy = x_train
    y_train_copy = y_train
    x_test_copy = x_test
    labels = []
    sample = []
    predictions = []
    for i in range(n_estimators):
        sample.append(np.random.choice(len(x_train), len(x_train),
                                       replace=True))
    for i in range(n_estimators):
        x_train = x_train_copy.copy()
        y_train = y_train_copy.copy()
        x_test_copy = x_test_copy.copy()
        
        x = x_train[sample[i]]
        y = y_train[sample[i]]
        tree = decision_tree(x, y)
        labels.append(predict(x_test_copy, tree))
    
    for index in range(len(labels[0])):
        prediction_dictionary = {}
        for tree_result in range(len(labels)):
            try:
                prediction_dictionary[labels[tree_result][index]] += 1
            except KeyError:
                prediction_dictionary[labels[tree_result][index]] = 1
        store = 0
        for index, value in prediction_dictionary.iteritems():
            if value > store:
                store = value
                chosen = index
        predictions.append(chosen)
        
    return predictions 

In [23]:
predictions = random_forest(x_train, y_train, x_test, n_estimators=100)

In [24]:
for i,v in enumerate(predictions):
    print v, y_test[i]

29.9 23.6
32.5 32.4
15.2 13.6
21.7 22.8
13.5 16.1
29.8 20.0
21.6 17.8
18.4 14.0
25.0 19.6
24.5 16.8
23.3 21.5
31.5 18.9
7.2 7.0
29.8 21.2
20.4 18.5
16.7 29.8
20.5 18.8
10.5 10.2
42.3 50.0
13.5 14.1
24.4 25.2
32.5 29.1
14.5 12.7
22.0 22.4
21.9 14.2
14.5 13.8
29.8 20.3
13.5 14.9
22.4 21.7
24.5 18.3
29.9 23.1
20.4 23.8
13.8 15.0
25.0 20.8
14.5 19.1
15.6 19.4
44.8 34.7
18.5 19.5
22.4 24.4
21.7 23.4
17.5 19.7
28.4 28.2
42.3 50.0
44.8 17.4
28.7 22.6
10.5 15.1
14.5 13.1
21.7 24.2
20.6 19.9
14.5 24.0
19.5 18.9
35.1 35.4
13.5 15.2
19.5 26.5
22.0 43.5
20.6 21.2
21.9 18.4
27.9 28.5
26.6 23.9
24.5 18.5
16.0 25.0
32.9 35.4
44.8 31.5
14.5 20.2
27.9 24.1
22.4 20.0
8.5 13.1
23.7 24.8
34.9 30.8
14.3 12.7
26.6 20.0
25.0 23.7
12.6 10.8
27.9 20.6
29.8 20.8
7.2 5.0
19.5 20.1
42.3 48.5
13.8 10.9
15.2 7.0
44.8 20.9
27.5 17.2
32.2 20.9
10.5 9.7
19.8 19.4
24.0 29.0
13.5 16.4
20.4 25.0
22.9 25.0
18.0 17.1
29.9 23.2
9.5 10.4
18.0 19.6
17.5 17.2
21.9 27.5
18.1 23.0
13.8 50.0
50.0 17.9
13.4 9.6
50.0 17.2
26.6 22.5

In [25]:
sse = 0
for i, v in enumerate(b):
    sse +=  (v-y_test[i])**2
sse

6696.2599999999975